# Data drift dashboard in jupyter notebook

In [ ]:
try:
    import evidently
except:
    !pip install git+https://github.com/evidentlyai/evidently.git

In [ ]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io

from datetime import datetime, time
from sklearn import datasets, ensemble

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, RegressionPreset

## Bicycle Demand Data

This step automatically downloads the bike dataset from UCI. This version is slightly different from the dataset used in Kaggle competition. If you want the example to be identical to the one in the Evidently blog "How to break a model in 20 days", you can manually download the dataset from Kaggle: https://www.kaggle.com/c/bike-sharing-demand/data 

In [ ]:
content = requests.get("https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip").content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday'], index_col='dteday')

In [ ]:
raw_data.index = raw_data.apply(
    lambda row: datetime.combine(row.name, time(hour=int(row['hr']))), axis = 1)

In [ ]:
raw_data.head()

## Regression Model

### Model training

In [ ]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']

In [ ]:
reference = raw_data.loc['2011-01-01 00:00:00':'2011-01-28 23:00:00']
current = raw_data.loc['2011-01-29 00:00:00':'2011-02-28 23:00:00']

In [ ]:
reference.head()

In [ ]:
regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)

In [ ]:
regressor.fit(reference[numerical_features + categorical_features], reference[target])

In [ ]:
ref_prediction = regressor.predict(reference[numerical_features + categorical_features])
current_prediction = regressor.predict(current[numerical_features + categorical_features])

In [ ]:
reference['prediction'] = ref_prediction
current['prediction'] = current_prediction

### Model Perfomance 

In [ ]:
column_mapping = ColumnMapping()

column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [ ]:
regression_perfomance = Report(metrics=[RegressionPreset()])
regression_perfomance.run(current_data=reference, reference_data=None, column_mapping=column_mapping)

In [ ]:
regression_perfomance.show()

In [ ]:
#regression_perfomance.save('reports/regression_performance_at_training.html')

##  Week 1

In [ ]:
regression_perfomance = Report(metrics=[RegressionPreset()])
regression_perfomance.run(current_data=current.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'], 
                          reference_data=reference,
                          column_mapping=column_mapping)

regression_perfomance.show()

In [ ]:
#regression_perfomance.save('reports/regression_performance_after_week1.html')

In [ ]:
target_drift = Report(metrics=[TargetDriftPreset()])
target_drift.run(current_data=current.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'],
                 reference_data=reference,
                 column_mapping=column_mapping)

target_drift.show()

In [ ]:
#target_drift.save('reports/target_drift_after_week1.html')

## Week 2

In [ ]:
regression_perfomance = Report(metrics=[RegressionPreset()])
regression_perfomance.run(current_data=current.loc['2011-02-07 00:00:00':'2011-02-14 23:00:00'], 
                          reference_data=reference,
                          column_mapping=column_mapping)

regression_perfomance.show()

In [ ]:
#regression_perfomance.save('reports/regression_performance_after_week2.html')

In [ ]:
target_drift = Report(metrics=[TargetDriftPreset()])
target_drift.run(current_data=current.loc['2011-02-07 00:00:00':'2011-02-14 23:00:00'],
                 reference_data=reference,
                 column_mapping=column_mapping)

target_drift.show()

In [ ]:
#target_drift.save('reports/target_drift_after_week2.html')

## Week 3

In [ ]:
regression_perfomance = Report(metrics=[RegressionPreset()])
regression_perfomance.run(current_data=current.loc['2011-02-15 00:00:00':'2011-02-21 23:00:00'], 
                          reference_data=reference,
                          column_mapping=column_mapping)

regression_perfomance.show()

In [ ]:
#regression_perfomance.save('reports/regression_performance_after_week3.html')

In [ ]:
target_drift = Report(metrics=[TargetDriftPreset()])
target_drift.run(current_data=current.loc['2011-02-15 00:00:00':'2011-02-21 23:00:00'],
                 reference_data=reference,
                 column_mapping=column_mapping)

target_drift.show()

In [ ]:
#target_drift.save('reports/target_drift_after_week3.html')

## Data Drift

In [ ]:
column_mapping = ColumnMapping()

column_mapping.numerical_features = numerical_features

In [ ]:
data_drift = Report(metrics = [DataDriftPreset()])
data_drift.run(current_data = current.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'],
               reference_data = reference,
               column_mapping=column_mapping)

data_drift.show()

In [ ]:
#data_drift.save("reports/data_drift_dashboard_after_week1.html")

# Support Evidently
Enjoyed the tutorial? Star Evidently on GitHub to contribute back! This helps us continue creating free open-source tools for the community. https://github.com/evidentlyai/evidently